# 0. Import

In [14]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import joblib

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# EDA
#import klib

# Preprocessing & Feature Engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier,Pool
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import ClassifierMixin
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Evaluation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold,KFold

# Utility
import os
import time
import random
import datetime as dt
import warnings; warnings.filterwarnings("ignore")
from datetime import datetime
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
#matplotlib 한글깨짐 지원
import platform

path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...')
rc('axes', unicode_minus=False)    

from itertools import combinations
from scipy.stats.mstats import gmean
from sklearn.feature_selection import RFE

import gc
import random
import re
from sklearn.metrics import *
import warnings
warnings.filterwarnings(action='ignore')

# import torch
# from torch import nn
# from pytorch_tabnet.tab_model import TabNetClassifier 
# from pytorch_tabnet.metrics import Metric

Unknown system...


# 1. Load Data

In [53]:
data_train = pd.read_csv('data_train.csv')
data_test = pd.read_csv('data_test.csv')

In [54]:
data_train.shape, data_test.shape

((10270011, 49), (3257159, 49))

In [56]:
data_train.drop_duplicates().shape

(10270001, 49)

# 2. Down Sampling

In [4]:
# 대용량의 데이터의 down-sampling을 위해 별도의 python 파일을 생성하여 터미널에서 작동.
# 파일은 별도 첨부하겠음.
# 코드는 아래와 같음.

__`Undersampling.py`__
```python
from tqdm import tqdm
from numpy import random
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import datetime as dt
if __name__ == "__main__":
    data_train = pd.read_csv('tr_feature_woo.csv',encoding='UTF-8')
    data_test = pd.read_csv('te_feature_woo.csv',encoding='UTF-8')##
    dat = data_train['신청서번호'].unique()
    a = data_train.query('신청서번호 in @dat').groupby('신청서번호')['신청서번호'].count()
    a = a.rename('신청수')
    a = a.reset_index()
    mini = data_train[['신청서번호','신청여부']]
    rand_idx = []
    index_rand = []
    z=0
    for j in tqdm(dat):
        rnd_num = []
        df = mini.query('신청서번호 == @j & 신청여부 == 0 ').index
        num = a.query('신청서번호 == @j')['신청수']
        if len(df) != 0:
            if len(df) > num.values[0]:
                if int(num.values[0]*0.85) > 5:
                    rnd_num = list(np.random.choice(len(df), size  =int(num.values[0]*0.85),replace = False))
                elif int(num.values[0]*0.85) > 3:
                    rnd_num = list(np.random.choice(len(df), size  =int(num.values[0]/2),replace = False))
                else:
                    rnd_num = list(np.random.choice(len(df), size  = 1 ,replace = False))
            else:
                rnd_num = [i for i in range(len(df))]
        else:
            rnd_num = [-1234]
        for idx in rnd_num:
            if idx == -1234:
                pass
            else:
                index_rand.append(df[idx])
    with open("index_final.npy", "wb") as fp:   #Pickling
        pickle.dump(index_rand, fp)
```

In [57]:
# Undersampling.py 에서 나온 index_final.npy 의 index 제거
idx = np.load('index_final.npy', allow_pickle=True)

In [58]:
data_train = data_train.drop(index = idx).reset_index(drop = True)

In [60]:
data_train.drop_duplicates().shape

(6061365, 49)

In [7]:
data_train.shape, data_test.shape

((6061374, 49), (3257159, 49))

# 3. Modeling

In [22]:
data_train.shape

(6061374, 49)

In [23]:
X_train.shape

(1120672, 32)

In [25]:
X_train = data_train[['유저번호','신청서번호','금융사번호', '상품번호', '승인한도', '승인금리', '유저성별', '한도조회당시유저신용점수', '연소득', '근로형태','입사연월', '고용형태', '주거소유형태', '대출희망금액', '대출목적', '개인회생자여부', '개인회생자납입완료여부','기대출수', '기대출금액', '한도조회월', '상품코드', '유저타입', '상품매력도', '연이자부담지수','3-5월평균순위변동폭', '6월예상순위', '연소득증감률', '과대출자', '기출등급', '나이', '앱버전','앱버전_세부내역', '신청서조회_비율', '신용점수_증감률']]
X_test = data_test[['유저번호','신청서번호','금융사번호', '상품번호', '승인한도', '승인금리', '유저성별', '한도조회당시유저신용점수', '연소득', '근로형태','입사연월', '고용형태', '주거소유형태', '대출희망금액', '대출목적', '개인회생자여부', '개인회생자납입완료여부','기대출수', '기대출금액', '한도조회월', '상품코드', '유저타입', '상품매력도', '연이자부담지수','3-5월평균순위변동폭', '6월예상순위', '연소득증감률', '과대출자', '기출등급', '나이', '앱버전','앱버전_세부내역', '신청서조회_비율', '신용점수_증감률']]


In [8]:
X_train = data_train[['금융사번호', '상품번호', '승인한도', '승인금리', '유저성별', '한도조회당시유저신용점수', '연소득', '근로형태','입사연월', '고용형태', '주거소유형태', '대출희망금액', '대출목적', '개인회생자여부', '개인회생자납입완료여부','기대출수', '기대출금액', '한도조회월', '상품코드', '유저타입', '상품매력도', '연이자부담지수','3-5월평균순위변동폭', '6월예상순위', '연소득증감률', '과대출자', '기출등급', '나이', '앱버전','앱버전_세부내역', '신청서조회_비율', '신용점수_증감률']]
X_test = data_test[['금융사번호', '상품번호', '승인한도', '승인금리', '유저성별', '한도조회당시유저신용점수', '연소득', '근로형태','입사연월', '고용형태', '주거소유형태', '대출희망금액', '대출목적', '개인회생자여부', '개인회생자납입완료여부','기대출수', '기대출금액', '한도조회월', '상품코드', '유저타입', '상품매력도', '연이자부담지수','3-5월평균순위변동폭', '6월예상순위', '연소득증감률', '과대출자', '기출등급', '나이', '앱버전','앱버전_세부내역', '신청서조회_비율', '신용점수_증감률']]
y_train = data_train['신청여부']

In [26]:
cat_features = ['금융사번호','상품번호','유저성별','근로형태','입사연월','고용형태','주거소유형태','대출목적','개인회생자여부','개인회생자납입완료여부','한도조회월','상품코드','유저타입','과대출자','기출등급','나이','앱버전','앱버전_세부내역','']
num_features = list(set(X_train.columns) - set(cat_features))

In [27]:
# y class들의 불균형을 잡아주기 위한 random_sampling
from sklearn.utils import resample
X=pd.concat([X_train,y_train],axis=1)


not_app=X[X.신청여부==0]
app=X[X.신청여부==1]



not_fraud_downsampled = resample(not_app,
                                replace = False, # sample without replacement
                                n_samples = int(len(app)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_fraud_downsampled, app])

# checking counts
downsampled.신청여부.value_counts()

0.0    560336
1.0    560336
Name: 신청여부, dtype: int64

In [28]:
X_train = downsampled[X_train.columns.tolist()]
y_train = downsampled['신청여부']

In [29]:
X_train.shape, y_train.shape, X_test.shape

((1120672, 34), (1120672,), (3257159, 34))

## - Catboost

In [54]:
is_holdout = False
n_splits = 5
iterations = 1000
patience = 50

cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
scv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [55]:
from sklearn.utils.class_weight import compute_class_weight
 
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = dict(zip(classes, weights))

In [56]:
gc.collect()

514

In [57]:
scores = []
models = []


models = []
#for tri, vai in cv.split(X_train):          #KFold
for tri, vai in scv.split(X_train,y_train): #StratifiedKFold
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=iterations,
                               random_state=42,
                               task_type="GPU",
                               eval_metric="F1",
                               cat_features=cat_features,
                               one_hot_max_size=4,
                               #learning_rate=0.05
                              )
    model.fit(X_train.iloc[tri], y_train.iloc[tri], 
            eval_set=[(X_train.iloc[vai], y_train.iloc[vai])], 
            early_stopping_rounds=patience ,
            use_best_model=True,
            verbose = 100
        )
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break 

Learning rate set to 0.039954
0:	learn: 0.7854368	test: 0.7884320	best: 0.7884320 (0)	total: 44.6ms	remaining: 44.6s
100:	learn: 0.8376331	test: 0.8407072	best: 0.8407072 (100)	total: 3.88s	remaining: 34.6s
200:	learn: 0.8486094	test: 0.8534652	best: 0.8534652 (200)	total: 7.62s	remaining: 30.3s
300:	learn: 0.8533182	test: 0.8586699	best: 0.8586936 (299)	total: 11.3s	remaining: 26.2s
400:	learn: 0.8564484	test: 0.8616904	best: 0.8617240 (399)	total: 14.9s	remaining: 22.3s
500:	learn: 0.8588787	test: 0.8643679	best: 0.8643679 (500)	total: 18.5s	remaining: 18.5s
600:	learn: 0.8609467	test: 0.8664596	best: 0.8665249 (598)	total: 22.2s	remaining: 14.7s
700:	learn: 0.8625730	test: 0.8683107	best: 0.8683107 (700)	total: 25.9s	remaining: 11s
800:	learn: 0.8640020	test: 0.8698139	best: 0.8698290 (797)	total: 29.6s	remaining: 7.35s
900:	learn: 0.8651148	test: 0.8708542	best: 0.8708542 (900)	total: 33.3s	remaining: 3.66s
999:	learn: 0.8659978	test: 0.8721012	best: 0.8721197 (997)	total: 36.9s	re

In [58]:
#f1 score에 대한 정리
#average의 default는 binary, binary인 경우 pos_label이라는 파라미터 값이 default로 1이 되는데 이경우 라벨값이 1인경우의 f1 score만을 산출함
#즉, 1과 0의 total f1 score가 아닌 1의 f1 score가 출력됨
#average="micro" : 일반적으로 우리가 아는 총합 정밀도와 재현율을 통해 구하는 f1 score
#average="macro" : label별로 가중을 두지 않고 각각 정밀도, 재현율, f1 score를 구해서 평균을 낸 정밀도, 재현율을 통해 f1 score를 출력함
#average="weighted" : macro인 경우에서 소수 label에 가중을 두고 구한 f1 score

#Catboost의 F1 score는 average = "micro" 인것으로 판단됨(일반적으로 우리가 아는 총합 f1 score 산출방식)

def score_function(real, pred):
    score = f1_score(real, pred, average="micro")
    return score

In [59]:
threshold = 0.4

In [62]:
preds = []
scores = []

#for i,(tri, vai) in enumerate(cv.split(X_train)):                #KFold
for i,(tri, vai) in enumerate(scv.split(X_train,y_train)):       #StratifiedKFold
#    pred = models[i].predict_proba(X_train.iloc[vai])[:, 1]
    pred = models[i].predict(X_train.iloc[vai])
#    pred = np.where(pred >= threshold , 1, 0)
    score = score_function(y_train.iloc[vai],pred)
    scores.append(score)
#    pred = models[i].predict_proba(X_test)[:, 1]
    preds.append(pred)
print(scores)
print(np.mean(scores))

[0.8688647466928414, 0.8688825930800633, 0.8679450685750488, 0.867057206849474, 0.8676862948057859]
0.8680871820006427


In [65]:
pred = np.where(pred >= threshold , 1, 0)

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_train.iloc[vai], pred, target_names=['class 0', 'class 1']))

              precision    recall  f1-score   support

     class 0       0.89      0.84      0.86    112067
     class 1       0.85      0.89      0.87    112067

    accuracy                           0.87    224134
   macro avg       0.87      0.87      0.87    224134
weighted avg       0.87      0.87      0.87    224134



## - LGBM

In [ ]:
lgbm = LGBMClassifier(n_estimators = 1000, random_state=0)

lgbm_model = lgbm.fit(X_train,y_train)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score_lgbm = cross_val_score(lgbm_model,X_train,y_train,scoring='f1',cv=skf,n_jobs=-1)

print(score_lgbm)
print(f'최대성능: {max(score_lgbm)}\n평균성능: {np.mean(score_lgbm)}')

In [ ]:
lgbm_model.fit(X_train, y_train)
joblib.dump(lgbm_model,'lgbm_model.pkl')

In [ ]:
gc.collect()

## - RandomForest

In [ ]:
# RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf_model = rf.fit(X_train,y_train)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score_rf = cross_val_score(rf_model,X_train,y_train,scoring='f1',cv=skf,n_jobs=-1)

print(score_rf)
print(f'최대성능: {max(score_rf)}\n평균성능: {np.mean(score_rf)}')

In [ ]:
rf_model.fit(X_train, y_train)
joblib.dump(rf_model,'rf_model.pkl')

## - DNN

In [ ]:
X_tr = X_train.copy()
X_te = X_test.copy()

In [ ]:
#StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_tr[num_features] = scaler.fit_transform(X_tr[num_features])
X_te[num_features] = scaler.transform(X_te[num_features])

In [ ]:
X_train_, X_dev, y_train_, y_dev = train_test_split(X_tr, y_train, test_size=0.4, random_state=42)

In [ ]:
# 매번 모델링을 할 때마다 동일한 결과를 얻기 위해 랜덤 시드 설정 동일하게 유지
# 럭키 시드 찾는건 시간상 어려움

# tensorflow 패키지
#!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
import re

def reset_seeds(reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(1)
    random.seed(2)
    tf.compat.v1.set_random_seed(3)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
    print("RANDOM SEEDS RESET")  # optional
   
reset_seeds()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

input = keras.Input(shape=(X_train_.shape[1],))

x = keras.layers.Dense(256, activation='relu')(input) # 은닉층 낮추기
output = keras.layers.Dense(1, activation='sigmoid')(x)

DEEP = keras.Model(input, output)

DEEP.summary()

#Image(keras.utils.model_to_dot(DEEP,show_shapes=True,show_layer_names=False, dpi=100).create(prog='dot', format='png'))

DEEP.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', keras.metrics.AUC()])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5),] # patience 낮추기

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

hist = DEEP.fit(X_train_, y_train_, validation_data=(X_dev, y_dev), 
                 batch_size=1024, epochs=50, callbacks=[callbacks, mc], shuffle=False, verbose=2)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
DEEP.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
history = DEEP.fit(X_train_, y_train_, validation_split=0.3, epochs=10, verbose=0)

# evaluate the model
loss, accuracy, f1_score, precision, recall = DEEP.evaluate(X_dev, y_dev, verbose=0)

print(f1_score)

In [ ]:
DEEP.fit(X_train, y_train) 

In [ ]:
#joblib.dump(DEEP,'dnn_model.pkl') 
DEEP.save('dnn_model.h5')

# 4. Submission

In [72]:
X_train.shape

(1120672, 32)

In [73]:
X_train = data_train.drop(columns = ['신청서번호','한도조회일시','유저번호','유저생년월일','생성일시','신청여부','한도조회월일','0','1','2','3','4','5','6','7','8','9'])
y_train = data_train['신청여부']
X_test = data_test.drop(columns = ['신청서번호','한도조회일시','유저번호','유저생년월일','생성일시','신청여부','한도조회월일','0','1','2','3','4','5','6','7','8','9'])
tr_application_id = data_train['신청서번호']
te_application_id = data_test['신청서번호']

In [74]:
X_train.shape

(6061374, 32)

In [75]:
# y class들의 불균형을 잡아주기 위한 random_sampling
from sklearn.utils import resample
X=pd.concat([X_train,y_train],axis=1)


not_app=X[X.신청여부==0]
app=X[X.신청여부==1]



not_fraud_downsampled = resample(not_app,
                                replace = False, # sample without replacement
                                n_samples = int(len(app)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_fraud_downsampled, app])

# checking counts
downsampled.신청여부.value_counts()

0.0    560336
1.0    560336
Name: 신청여부, dtype: int64

In [76]:
X_train = downsampled[X_train.columns.tolist()]
y_train = downsampled['신청여부']

In [ ]:
import joblib

cat_model = joblib.load('models/cat_model.pkl')
lgbm_model = joblib.load('models/lgbm_model.pkl')
rf_model = joblib.load('models/rf_model.pkl')

import tensorflow as tf

dnn_model = tf.keras.models.load_model('best_model.h5')

cat_features =  ['금융사번호','상품번호','유저성별','근로형태','입사연월','고용형태','주거소유형태','대출목적','개인회생자여부','개인회생자납입완료여부','한도조회월','상품코드','유저타입','과대출자','기출등급','나이','mp_app_version','detail']
num_features = list(set(X_train.columns) - set(cat_features))


preds = []
pred_cat = cat_model.predict_proba(X_test)[:,1]
preds.append(pred_cat)

from sklearn.preprocessing import LabelEncoder

for i in cat_features:
    X_train[i] = LabelEncoder().fit_transform(X_train[i])

for i in cat_features:
    X_test[i] = LabelEncoder().fit_transform(X_test[i])

pred_lgb = lgbm_model.predict_proba(X_test)[:,1]
preds.append(pred_lgb)

#pred_rf = rf_model.predict_proba(X_test)[:.1]
#preds.append(pred_rf)

from sklearn.preprocessing import StandardScaler
X_tr = X_train.copy()
X_te = X_test.copy()
scaler = StandardScaler()
X_tr[num_features] = scaler.fit_transform(X_tr[num_features])
X_te[num_features] = scaler.transform(X_te[num_features])

pred_dnn = dnn_model.predict(X_te).flatten()
preds.appned(pred_dnn)

threshold = 0.5
preds = np.mean(preds , axis = 0 )
pred = np.where(preds > threshold,1,0)

In [ ]:
# test 데이터에 신청여부 predict 값 concat
a = pd.DataFrame({'신청여부' : pred})
data_test = data_test.drop(columns = '신청여부')
data_test = pd.concat([data_test, a], axis = 1)

submission = pd.read_csv('데이터분석분야_퓨처스부문_평가데이터.csv')

submission_test = data_test[['신청서번호','상품번호','신청여부']]
submission_test.rename(columns={'신청서번호':'application_id','상품번호':'product_id','신청여부':'is_applied_new'},inplace=True)

submission = pd.merge(submission, submission_test, how='left', on=['application_id','product_id'])

submission.drop(columns=['is_applied'],inplace=True)
submission.rename(columns={'is_applied_new':'is_applied'},inplace=True)

submission.to_csv('평가데이터정답지.csv', index=False)

### 모델 해석

In [ ]:
#Catboost
model = joblib.load('cat_model.pkl')

predictors = ['금융사번호', '상품번호', '승인한도', '승인금리', '유저성별', '한도조회당시유저신용점수', '연소득', '근로형태',
       '입사연월', '고용형태', '주거소유형태', '대출희망금액', '대출목적', '개인회생자여부', '개인회생자납입완료여부',
       '기대출수', '기대출금액', '한도조회월', '상품코드', '유저타입', '상품매력도', '연이자부담지수',
       '3-5월평균순위변동폭', '6월예상순위', '연소득증감률', '과대출자', '기출등급', '나이', '앱버전',
       '앱버전_세부내역', '신청서조회_비율', '신용점수_증감률']

tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': model.feature_importances_})
tmp = tmp.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (7,4))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()  

In [ ]:
#LGBM
import joblib

model = joblib.load('lgbm_model.pkl')

tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': model.feature_importances_})
tmp = tmp.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (7,4))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()

In [ ]:
#RandomForest
model = joblib.load('rf_model.pkl')

tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': model.feature_importances_})
tmp = tmp.sort_values(by='Feature importance',ascending=False)
plt.figure(figsize = (7,4))
plt.title('Features importance',fontsize=14)
s = sns.barplot(x='Feature',y='Feature importance',data=tmp)
s.set_xticklabels(s.get_xticklabels(),rotation=90)
plt.show()

# 5. K-Means

In [12]:
gc.collect()

45

In [13]:
X = pd.concat([X_train, X_test], axis = 0)

In [16]:
# StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X[num_features])
#X_test[num_features] = scaler.transform(X_test[num_features])

In [141]:
from sklearn.cluster import KMeans

k = 3
# 그룹 수, random_state 설정
model = KMeans(n_clusters = k, random_state = 10)
# 정규화된 데이터에 학습
model.fit(scaled_X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=10, tol=0.0001, verbose=0)

In [142]:
X['cluster'] = model.labels_

In [144]:
X = X.reset_index(drop = True)

In [145]:
target = pd.concat([data_test['신청여부'], downsampled['신청여부']], axis = 0).reset_index(drop=True)

In [148]:
X = pd.concat([X, target], axis = 1)

In [154]:
X.to_csv('data/for_EDA.csv', index = False)